In [1]:
import folium
import geopandas as gpd
import pickle as pkl

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Adding Datasets

In [27]:
# importing datasets

# LANDUSE DATA
brownfields = gpd.read_file('/content/drive/MyDrive/FinalProject/Data/Landuse-data/Christchurch/brownfields.gpkg')

built_up_areas = gpd.read_file('/content/drive/MyDrive/FinalProject/Data/Landuse-data/Christchurch/built_up_areas.gpkg')

public_parks = gpd.read_file('/content/drive/MyDrive/FinalProject/Data/Landuse-data/Christchurch/public-parks.gpkg')

# Population Density
population_density = gpd.read_file('/content/drive/MyDrive/FinalProject/Data/Pop_Density_Data/Chch/processed/pop_density.gpkg')

# Hospital locations
hospitals = gpd.read_file('/content/drive/MyDrive/FinalProject/Data/amenities/Christchurch/hospitals.gpkg')

# Isochrones
with open('/content/drive/MyDrive/FinalProject/Data/amenity-isochrones/christchurch_amenity_isochrones.pkl', 'rb') as file:
  isochrones = pkl.load(file)

# NDVI greenspace
green_ndvi = gpd.read_file('/content/drive/MyDrive/FinalProject/Data/Landuse-data/Christchurch/ndvi-quality.gpkg')

In [28]:
m = folium.Map(location=[-43.5299, 172.6333], zoom_start=12, tiles='cartodb positron')


## Built up Area

In [29]:
def style_function(feature):
    return {
        "fillColor": "#6D676E",  # Fill color of the buildings
        "color": "#6D676E",    # Border color
        "weight": 1,         # Border width
        "radius": 8,         # Size of the markers
        "fillOpacity": 1,    # Opacity of the markers
    }

folium.GeoJson(
    built_up_areas,
    name="High Density Building Coverage",
    control=True,
    style_function=style_function,
    show=True
).add_to(m)

## Population Density

In [30]:
def style_function(x):
    density = x['population_density']  # Adjust the column name as per your GeoDataFrame
    return {
        'fillColor': 'YlGnBu',  # Choose a color scale (e.g., 'YlGnBu', 'YlOrRd', 'RdYlBu', etc.)
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7 if density is not None else 0,
    }

# mapping population density per statistical area
population = folium.Choropleth(
    geo_data=population_density,
    data=population_density,
    columns=['SA12018_V1','Pop_density'],
    key_on='feature.properties.SA12018_V1',
    fill_color='YlGnBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Population Density per square km',
    highlight=True,
    name='Population Density',
    show=True,
    control=True,
    style_function=style_function
).add_to(m)


## Public Parks

In [31]:
# Creating a dictionary of colours to represent greenspace quality
category_colours = {
        'Low': '#94C58C',
        'Medium': '#1A8828',
        'High': '#094F29',
    }

# map using geojson
folium.GeoJson(green_ndvi,
               style_function=lambda x: {
        'fillColor': category_colours.get(x['properties']['Greenspace_Quality'], 'gray'),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7,
    },
    name="Public Greenspace",
    control=True,
    tooltip=folium.GeoJsonTooltip(fields=["name", 'Greenspace_Quality', 'Average_NDVI'],
                                  aliases=["Park Name", 'Greenspace Quality', 'Average NDVI Value'],
                                  labels=True,
                                  localize=True),
    show=False
).add_to(m)


## Hospitals

In [32]:
def style_function(feature):
    return {
        "fillColor": "red",  # Fill color of the hospital markers
        "color": "white",    # Border color of the hospital markers
        "weight": 1,         # Border width
        "radius": 8,         # Size of the markers
        "fillOpacity": 1,    # Opacity of the markers
    }

folium.GeoJson(
    hospitals,
    name="Hospitals",
    control=True,
    tooltip=folium.GeoJsonTooltip(fields=["name"], aliases=["Hospital"], labels=True),
    popup=f"{hospitals['name']}",
    style_function=style_function,
    show=False
).add_to(m)

## Brownfields

In [33]:
def style_function(feature):
    return {
        "fillColor": "#A0522D",  # Fill color of the brownfields markers
        "color": "black",    # Border color
        "weight": 1,         # Border width
        "radius": 8,         # Size of the markers
        "fillOpacity": 1,    # Opacity of the markers
    }

folium.GeoJson(
    brownfields,
    name="Brownfields",
    tooltip='Brownfield',
    popup=f"{brownfields['description']}",
    style_function=style_function,
    show=False
).add_to(m)

## Isochrones

In [34]:
# Visualising Isochrone data

# create a layer for schools
sch_layer = folium.FeatureGroup(name='Schools', show=False)

# create a layer for supermarkets
sup_layer = folium.FeatureGroup(name='Supermarkets', show=False)

# Create polygons showing the isochrone boundaries
for key in isochrones.keys():

    if 'isochrone' in isochrones[key] and isochrones[key]['amenity'] == 'supermarket':
    # Add isochrone to the map
        folium.features.GeoJson(
            data=isochrones[key]['isochrone']
            , style_function=lambda feature: dict(color='#2C7329')
        ).add_to(sup_layer)

    elif 'isochrone' in isochrones[key] and isochrones[key]['amenity'] == 'school':
    # Add isochrone to the map
        folium.features.GeoJson(
            data=isochrones[key]['isochrone']
            , style_function=lambda feature: dict(color='#3171AE')
        ).add_to(sch_layer)

    else:
        continue

# Add a marker to the centre of each isochrone
for key in isochrones.keys():

  if 'isochrone' in isochrones[key] and isochrones[key]['amenity'] == 'school':

      # Add a school marker to map:
      folium.map.Marker(
          isochrones[key]['location'],
          icon=folium.Icon(color='white',
                            icon_color='#3171AE',
                            icon='graduation-cap',
                            prefix='fa'
                          ),
          popup=f"{isochrones[key]['display name']}"
      ).add_to(sch_layer)

  elif 'isochrone' in isochrones[key] and isochrones[key]['amenity'] == 'supermarket':

      # supermarket marker
      folium.map.Marker(
          isochrones[key]['location'],
          icon=folium.Icon(color='white',
                            icon_color='#2C7329',
                            icon='shopping-cart',
                            prefix='fa'
                          ),
          popup=f"{isochrones[key]['display name']}"
      ).add_to(sup_layer)


# add the layers to the map
sch_layer.add_to(m)
sup_layer.add_to(m)

## Map visualisation

In [35]:
folium.LayerControl(collapsed=False).add_to(m)

m

Output hidden; open in https://colab.research.google.com to view.

## Saving Map

In [36]:
m.save('/content/drive/MyDrive/FinalProject/Maps/Christchurch_liveability_map.html')